In [2]:
import random as rand
import math
import copy

Especificaciones

In [3]:
clientes = [
    "FreshMart", "EcoStore", "QuickShop", "DailyMart", "GreenWay", 
    "ValueStop", "EasyGrocer", "MegaFresh", "FoodHaven", "BudgetBazaar",
    "UrbanMarket", "HappyGrocer", "PrimeFoods", "SmartMart", "GrocerTown",
    "PureFoods", "LocalBasket", "NatureNest", "QuickCart", "SaveMore",
    "FarmPick", "CityGrocer", "MarketPlace", "ChoiceMart", "GoGrocer",
    "FreshNest", "HarvestMart", "BrightBasket", "DailyChoice", "TopMart"
]

In [4]:
def normalizar(lista):
    suma = sum(lista)
    for i in range(len(lista)):
        lista[i] = lista[i]/suma
    return lista

In [5]:
def nivel_cliente():
    nivel = rand.randrange(1,4)
    return(nivel)

In [6]:
def tiempo_cliente():
    tiempo = rand.randrange(1,1000)
    return(tiempo)

In [7]:
lista_nivel = []
lista_tiempo = []
for i in range(len(clientes)):
    lista_nivel.append(nivel_cliente())
    lista_tiempo.append(tiempo_cliente())
lista_nivel = normalizar(lista_nivel)
lista_tiempo = normalizar(lista_tiempo)

In [8]:
tabla_clientes = []
for i in range(len(clientes)):
    tabla_clientes.append([clientes[i], lista_nivel[i], lista_tiempo[i]])

In [9]:
dificutlad = [2,3,2,3,3,4]
precio = [60,100,50,80,80,130]
lista_dificultad = normalizar(dificutlad)
lista_precio = normalizar(precio)
tabla_productos = []
for i in range(1,7):
    tabla_productos.append(["P"+str(i), lista_dificultad[i-1], lista_precio[i-1]])

In [11]:
productos = [
    "P1_500ml","P2_1000ml","P3_500ml","P4_1000ml","P5_500ml","P6_1000ml",
]

In [12]:
cantidad_productos = [40,80,120,160,200,240,280,320,360,400]

In [13]:
def generar_pedido_aleatorio():
    
    cliente = rand.choices(clientes, k = 1)[0]
    producto_botella = rand.choices(productos, weights = [0.20,0.15,0.2,0.15,0.2,0.15], k = 1)[0]
    producto = producto_botella.split("_")[0]
    botella = producto_botella.split("_")[1]
    cantidad = rand.choices(cantidad_productos, weights = [0.5,0.3,0.1,0.03,0.02,0.01,0.01,0.01,0.01,0.01], k = 1)[0]
    return [cliente,f"{producto}*{cantidad}_{botella}"]

Modelo Matematico

In [14]:
# Parámetros ajustables
beta1 = 1 #Entero: peso de la función de valor de cliente
beta2 = 1 #Entero: peso de la función de valor de producto 
alpha1 = 1 #Entero: peso del nivel del cliente
alpha2 = 1 #Entero: peso del tiempo del cliente
lambda1 = 1 #Entero: peso de la cantidad de producto
lambda2 = 1 #Entero: peso del precio del producto
lambda3 = 1 #Entero: peso de la dificultad del producto
parametros = [beta1, beta2, alpha1, alpha2, lambda1, lambda2, lambda3]

In [10]:
def validarParametros(parametros):
    for parametro in parametros:
        if parametro < 0:
            return "Error: Todos los parámetros deben ser mayores o iguales a 0"
        elif parametro > 10:
            return "Error: Los parámetros no pueden ser mayores a 10"
        elif not isinstance(parametro, int):
            return "Error: Los parámetros deben ser números enteros"
    else:
        return True

In [15]:
validacion = validarParametros(parametros)
if validacion == True:
    pedidos_a_simular = [generar_pedido_aleatorio() for i in range(26)]
    pedidos_a_simular
else:
    print(validacion)

In [16]:
def valor_de_pedido(pedido,parametros,cantidad_total):
    producto = (pedido[1].split("_")[0]).split("*")[0]
    cant = int((pedido[1].split("_")[0]).split("*")[1])
    cant_norm = cant/cantidad_total
    clien = pedido[0]
    for i in range (len(tabla_productos)):
        if tabla_productos[i][0]==producto:
            dif = tabla_productos[i][1]
            prec = tabla_productos[i][2]
            break
    for i in range (len(tabla_clientes)):
        if tabla_clientes[i][0]==clien:
            niv = tabla_clientes[i][1]
            tie = tabla_clientes[i][2]
            break
    b1 = parametros[0]
    b2 = parametros[1]
    a1 = parametros[2]
    a2 = parametros[3]
    l1 = parametros[4]
    l2 = parametros[5]
    l3 = parametros[6]
    return (10*b1*(a1*niv + a2*tie) + b2*(l1*cant_norm + 2*l2*prec + l3/2*(1 -abs(dif))))

In [17]:
def pedidos_a_datos(pedidos):
    param = parametros
    cantidad_total = 0
    for pedido in pedidos:
    
        cantidad_total += int((pedido[1].split("_")[0]).split("*")[1])
    for pedido in pedidos:
    
        valor = valor_de_pedido(pedido, param, cantidad_total)
        
        pedido.insert(0,valor)
        
    pedidos_ordenados = sorted(pedidos, reverse = True)
    
    for idx, pedido in enumerate(pedidos_ordenados):
        
         pedido.insert(0,f"P{idx+1}")
            
    return pedidos_ordenados

In [18]:
pedidos_a_simular = pedidos_a_datos(pedidos_a_simular)

Interface

In [19]:
def despliegue_de_maquinas(maquinas, descompuestas = []):
    print("-" * 101)
    for idx, maquina in enumerate(maquinas):
        if idx+1 not in descompuestas:
            row = f"M{idx+1}  |" + "|".join([str(slot) for slot in maquina])
            print(f"{row}|")
        else:
            row = f"M{idx+1}  | maquina descompuesta"
            print(row)
        print("-" * 101)

## Reprogramacion 1: Eliminar Pedido

In [20]:
def eliminar_pedido(maquinas, pedido):
    for i in range(len(maquinas)):
        maquinas[i] = [" " if item == pedido else item for item in maquinas[i]]
    return maquinas

## Reprogramacion 2: Descomponer Maquina

In [131]:
def descomponer_maquina(maquinas, maq, descompuestas):
    if maq not in descompuestas:
        descompuestas.append(maq)
    pedidos_a_reprogramar = list(set(maquinas[maq-1]) - {' '}) 
    if pedidos_a_reprogramar:
        for p in pedidos_a_reprogramar:
            maquinas = eliminar_pedido(maquinas, p) 
    return maquinas, descompuestas, pedidos_a_reprogramar

Funciones Generales

In [22]:
def maquinas_posibles(_pedido):
    tam = _pedido.split("_")[1]
    if tam == "500ml":
        return [3,5]
    else:
        return [4,5]    

In [23]:
def litros_pedido(_pedido):
    listz = (_pedido.split("*")[1]).split("_")   
    litros = int(listz[0])*int(listz[1].split("ml")[0])/1000
    return int(litros)

In [24]:
def datos_pedido(_pedido):
    posibles = maquinas_posibles(_pedido)
    litros = litros_pedido(_pedido)
    return posibles, litros

## Reprogramacion 3: Agregar Pedido a Espacio

In [37]:
def slots_subfunction(maquina, slots, minimum_start):
    blank_counter = 0
    for idx in range(minimum_start, len(maquina)):  # Start iterating from minimum_start
        if maquina[idx] == " ":
            blank_counter += 1
            if blank_counter == slots:
                return idx - slots + 1, idx
        else:
            blank_counter = 0  # Reset counter if a slot is occupied

    return False, False

In [55]:
def empty_slots_bottle(maquina, cook_id, bottle_id, litros, prod, end_of_cook):
    check_id_list = []  # List to store reserved slots for bottling
    batch_size = 80 if cook_id == 2 else 40  # Cooking batch size (80L for M3)
    cook_length = 5 if cook_id == 2 else 4  # Cooking time per batch (M3 = 5 minutes per batch)
    
    if bottle_id == 3:
        bottle_capacity = 40
    elif bottle_id == 4:
        bottle_capacity = 80
    elif bottle_id == 5:
        if prod in ["P1","P3","P5"]:
            bottle_capacity = 40
        else:
            bottle_capacity = 80
    
    remaining_litros = litros
    current_cooking_end = end_of_cook - (math.ceil(litros / batch_size) - 1) * cook_length  # First batch cooking finish time

    while remaining_litros > 0:
        batch_litros = min(batch_size, remaining_litros)  # Liters for the current cooking batch
        remaining_litros -= batch_litros

        bottle_process_start = current_cooking_end + 1

        while batch_litros > 0:
            process_litros = min(bottle_capacity, batch_litros)  # Liters for this bottling process
            batch_litros -= process_litros
            
            process_start = bottle_process_start
            if bottle_id == 3:
                step = 1
            elif bottle_id == 4:
                step = 2
            elif bottle_id == 5:
                if prod in ["P1","P3","P5"]:
                    step = 2
                else:
                    step = 3
            process_end = process_start + step

            check_id_list.extend(range(process_start, process_end + 1))  # Reserve these IDs
            
            bottle_process_start = process_end + 1
        
        current_cooking_end += cook_length
    
    for idx in check_id_list:
        try:
            if maquina[idx] != " ":
                return False
        except:
            return False
    return check_id_list


In [29]:
def empty_slots_cook(maquina, maquina_id, litros, minimum=0):
    blank_counter = 0
    
    if maquina_id == 0 or maquina_id == 1:
        slots = max(4, 4 * math.ceil(litros / 40))
        start_id, end_id = slots_subfunction(maquina, slots, minimum)
        if start_id is not False:
            return start_id, end_id
                
    if maquina_id == 2:
        slots = max(5, 5 * math.ceil(litros / 80))
        start_id, end_id = slots_subfunction(maquina, slots, minimum)
        if start_id is not False:
            return start_id, end_id
        
    return False, False

In [157]:
def try_add(maquinas, posibles, litros, nombre, prod, descompuestas):
    minimum_start = 0 

    while minimum_start <= 42:
        cook_start_ids = []
        cook_end_ids = []

        for i in range(3):
            
            if i+1 in descompuestas:
                start_id, end_id = False, False
            else:  
                start_id, end_id = empty_slots_cook(maquinas[i], i, litros, minimum_start)
            cook_start_ids.append(start_id)
            cook_end_ids.append(end_id)
        
        if all(start_id is False for start_id in cook_start_ids):
            print("No valid cooking slots found. Exiting.")
            return maquinas

        tried_bottles = False
        while any(start_id is not False for start_id in cook_start_ids):
            min_cook_end_index = min(
                (idx for idx, end in enumerate(cook_end_ids) if end is not False),
                key=lambda idx: cook_end_ids[idx]
            )
            min_cook_start_id = cook_start_ids[min_cook_end_index]
            min_cook_end_id = cook_end_ids[min_cook_end_index]
            cook_machine = min_cook_end_index
            
            bottle_processes = []
            bottle_end_ids = []
            
            for j in posibles:
                if j in descompuestas:
                    end_id = False
                else:
                    bottle_id_list = empty_slots_bottle(maquinas[j], cook_machine, j, litros, prod, min_cook_end_id)
                    try:
                        end_id = bottle_id_list[-1]
                    except:
                        end_id = False
                    bottle_processes.append(bottle_id_list)
                    bottle_end_ids.append(end_id)

            if all(end_id is False for end_id in bottle_end_ids):
                cook_start_ids[min_cook_end_index] = False
                cook_end_ids[min_cook_end_index] = False
                if tried_bottles:
                    break
                tried_bottles = True
                continue

            min_bottle_end_index = min(
                (idx for idx, end in enumerate(bottle_end_ids) if end is not False),
                key=lambda idx: bottle_end_ids[idx]
            )
            
            min_bottle_end_id = bottle_end_ids[min_bottle_end_index]
            bottle_machine = posibles[min_bottle_end_index]
            final_bottle_ids = bottle_processes[min_bottle_end_index]
            
            print(f"Cooking: M{cook_machine+1} ({min_cook_start_id}, {min_cook_end_id})")
            print(f"Bottling: M{bottle_machine+1} ({final_bottle_ids})")

            for i in range(min_cook_start_id, min_cook_end_id + 1):
                maquinas[cook_machine][i] = nombre

            for j in final_bottle_ids:
                maquinas[bottle_machine][j] = nombre

            return maquinas 

        minimum_start += 1

    print("No valid slots found after exhausting all options.")
    return maquinas


In [158]:
def agregar_pedido(maquinas, pedido, waitlist):
    maquina_og = copy.deepcopy(maquinas)
    maquina_previo = copy.deepcopy(maquinas)
    posibles, litros = datos_pedido(pedido[3])
    codigo_pedido = list(filter(lambda x: pedidos_dict[x] == pedido, pedidos_dict))[0]
    print(codigo_pedido, pedido)
    print(litros,"Litros")
    prod = pedido[3].split("*")[0]
    maquina_post = try_add(maquina_previo, posibles, litros, codigo_pedido, prod)
    if maquina_post == maquina_og:
        print(f"\nNo se logro agregar {pedido} ({codigo_pedido})\n")
        wait_list.append(pedido)
    else:
        despliegue_de_maquinas(maquina_post)
        
    return maquinas, waitlist

## SIMULACION 1: Programacion Inicial

In [159]:
import string

uppercase_alphabet = string.ascii_uppercase

In [160]:
pedidos_dict = {}

for idx, pedido in enumerate(pedidos_a_simular):
    pedidos_dict[uppercase_alphabet[idx]] = pedido
    
pedidos_dict

{'A': ['P1', 1.6520154424108469, 'SaveMore', 'P5*40_500ml'],
 'B': ['P2', 1.6501433675278516, 'BrightBasket', 'P4*40_1000ml'],
 'C': ['P3', 1.6369544330072212, 'DailyMart', 'P2*400_1000ml'],
 'D': ['P4', 1.6155503046609518, 'NatureNest', 'P6*40_1000ml'],
 'E': ['P5', 1.5942105419795354, 'HappyGrocer', 'P6*40_1000ml'],
 'F': ['P6', 1.575262263192769, 'DailyChoice', 'P2*40_1000ml'],
 'G': ['P7', 1.5335572067552175, 'GreenWay', 'P6*40_1000ml'],
 'H': ['P8', 1.5335572067552175, 'GreenWay', 'P6*40_1000ml'],
 'I': ['P9', 1.5084265344444703, 'PrimeFoods', 'P5*80_500ml'],
 'J': ['P10', 1.4990239465762496, 'EasyGrocer', 'P1*80_500ml'],
 'K': ['P11', 1.4976537470874192, 'CityGrocer', 'P3*40_500ml'],
 'L': ['P12', 1.4698001537993024, 'QuickCart', 'P5*40_500ml'],
 'M': ['P13', 1.4667397760650442, 'BudgetBazaar', 'P5*80_500ml'],
 'N': ['P14', 1.4161515407709264, 'BudgetBazaar', 'P1*80_500ml'],
 'O': ['P15', 1.3842455672057807, 'GreenWay', 'P5*80_500ml'],
 'P': ['P16', 1.3531691241480586, 'LocalBask

In [161]:
m1 = [" "] * 48
m2 = [" "] * 48
m3 = [" "] * 48
m4 = [" "] * 48
m5 = [" "] * 48
m6 = [" "] * 48

sim_machines = [m1, m2, m3, m4, m5, m6]

In [162]:
import copy

wait_list = []
descompuestas = []
maquina_post = sim_machines

for pedido in pedidos_a_simular:
    maquina_og = copy.deepcopy(maquina_post)  # Deep copy
    maquina_previo = copy.deepcopy(maquina_post)  # Deep copy
    posibles, litros = datos_pedido(pedido[3])
    codigo_pedido = list(filter(lambda x: pedidos_dict[x] == pedido, pedidos_dict))[0]
    print(codigo_pedido, pedido)
    print(litros,"Litros")
    prod = pedido[3].split("*")[0]
    maquina_post = try_add(maquina_previo, posibles, litros, codigo_pedido, prod, descompuestas)
    if maquina_post == maquina_og:
        print(f"\nNo se logro agregar {pedido} ({codigo_pedido})\n")
        wait_list.append(pedido)
    else:
        despliegue_de_maquinas(maquina_post)


A ['P1', 1.6520154424108469, 'SaveMore', 'P5*40_500ml']
20 Litros
Cooking: M1 (0, 3)
Bottling: M4 ([4, 5])
-----------------------------------------------------------------------------------------------------
M1  |A|A|A|A| | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |
-----------------------------------------------------------------------------------------------------
M2  | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |
-----------------------------------------------------------------------------------------------------
M3  | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |
-----------------------------------------------------------------------------------------------------
M4  | | | | |A|A| | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |
-----------------------------------------------------------------------------

Pruebas Reprogramacion

* maquinas = eliminar_pedido(maquinas, pedido)
* agregar_pedido(maquinas, pedido, waitlist):
* maquinas_futuras = try_add(maquinas, posibles, litros, nombre, prod)

Consultamos la fila de espera

In [163]:
wait_list

[['P26', 1.057400355133559, 'SmartMart', 'P1*40_500ml']]

Comenzamos el dia con esta estructura

In [164]:
 despliegue_de_maquinas(maquina_post)

-----------------------------------------------------------------------------------------------------
M1  |A|A|A|A|D|D|D|D|E|E|E|E|F|F|F|F|G|G|G|G|H|H|H|H|N|N|N|N|R|R|R|R|R|R|R|R|T|T|T|T|X|X|X|X| | | | |
-----------------------------------------------------------------------------------------------------
M2  |B|B|B|B|I|I|I|I|J|J|J|J|K|K|K|K|L|L|L|L|M|M|M|M|P|P|P|P|P|P|P|P|S|S|S|S|U|U|U|U|Y|Y|Y|Y| | | | |
-----------------------------------------------------------------------------------------------------
M3  |C|C|C|C|C|C|C|C|C|C|C|C|C|C|C|C|C|C|C|C|C|C|C|C|C|O|O|O|O|O|Q|Q|Q|Q|Q|V|V|V|V|V|W|W|W|W|W| | | |
-----------------------------------------------------------------------------------------------------
M4  | | | | |A|A| | |I|I| | |J|J| | |K|K| | |L|L| | |M|M| | |N|N|O|O|R|R| | |R|R| | |U|U| | |X|X| | |
-----------------------------------------------------------------------------------------------------
M5  | | | | |B|B|B| |D|D|D| |E|E|E| |F|F|F| |G|G|G| |H|H|H| |P|P|P| |P|P|P|Q|Q|Q| 

Cancelan el Pedido C

In [165]:
maquina_post_eliminar_producto = eliminar_pedido(maquina_post, "C")
despliegue_de_maquinas(maquina_post_eliminar_producto)

-----------------------------------------------------------------------------------------------------
M1  |A|A|A|A|D|D|D|D|E|E|E|E|F|F|F|F|G|G|G|G|H|H|H|H|N|N|N|N|R|R|R|R|R|R|R|R|T|T|T|T|X|X|X|X| | | | |
-----------------------------------------------------------------------------------------------------
M2  |B|B|B|B|I|I|I|I|J|J|J|J|K|K|K|K|L|L|L|L|M|M|M|M|P|P|P|P|P|P|P|P|S|S|S|S|U|U|U|U|Y|Y|Y|Y| | | | |
-----------------------------------------------------------------------------------------------------
M3  | | | | | | | | | | | | | | | | | | | | | | | | | |O|O|O|O|O|Q|Q|Q|Q|Q|V|V|V|V|V|W|W|W|W|W| | | |
-----------------------------------------------------------------------------------------------------
M4  | | | | |A|A| | |I|I| | |J|J| | |K|K| | |L|L| | |M|M| | |N|N|O|O|R|R| | |R|R| | |U|U| | |X|X| | |
-----------------------------------------------------------------------------------------------------
M5  | | | | |B|B|B| |D|D|D| |E|E|E| |F|F|F| |G|G|G| |H|H|H| |P|P|P| |P|P|P|Q|Q|Q| 

Se busca agregar los productos en fila de espera

for pedido in wait_list:
    print(pedido)
    maquinas_post_agregar_fila_espera, wait_list = (maquina_post_eliminar_producto, pedido[3], wait_list)
    despliegue_de_maquinas(maquina_post_eliminar_producto)

In [166]:
maquina_post =maquina_post_eliminar_producto

for pedido in wait_list:
    maquina_og = copy.deepcopy(maquina_post)  # Deep copy
    maquina_previo = copy.deepcopy(maquina_post)  # Deep copy
    posibles, litros = datos_pedido(pedido[3])
    codigo_pedido = list(filter(lambda x: pedidos_dict[x] == pedido, pedidos_dict))[0]
    print(codigo_pedido, pedido)
    print(litros,"Litros")
    prod = pedido[3].split("*")[0]
    maquina_post = try_add(maquina_previo, posibles, litros, codigo_pedido, prod, descompuestas)
    if maquina_post == maquina_og:
        print(f"\nNo se logro agregar {pedido} ({codigo_pedido})\n")
        wait_list.append(pedido)
    else:
        despliegue_de_maquinas(maquina_post)

Z ['P26', 1.057400355133559, 'SmartMart', 'P1*40_500ml']
20 Litros
Cooking: M3 (0, 4)
Bottling: M6 ([5, 6, 7])
-----------------------------------------------------------------------------------------------------
M1  |A|A|A|A|D|D|D|D|E|E|E|E|F|F|F|F|G|G|G|G|H|H|H|H|N|N|N|N|R|R|R|R|R|R|R|R|T|T|T|T|X|X|X|X| | | | |
-----------------------------------------------------------------------------------------------------
M2  |B|B|B|B|I|I|I|I|J|J|J|J|K|K|K|K|L|L|L|L|M|M|M|M|P|P|P|P|P|P|P|P|S|S|S|S|U|U|U|U|Y|Y|Y|Y| | | | |
-----------------------------------------------------------------------------------------------------
M3  |Z|Z|Z|Z|Z| | | | | | | | | | | | | | | | | | | | |O|O|O|O|O|Q|Q|Q|Q|Q|V|V|V|V|V|W|W|W|W|W| | | |
-----------------------------------------------------------------------------------------------------
M4  | | | | |A|A| | |I|I| | |J|J| | |K|K| | |L|L| | |M|M| | |N|N|O|O|R|R| | |R|R| | |U|U| | |X|X| | |
-------------------------------------------------------------------------

Rompe maquina

In [167]:
descompuestas = []

In [168]:
maquina_post_romper, descompuestas, pedidos_a_reprogramar = descomponer_maquina(maquina_post, 1, descompuestas)

In [169]:
despliegue_de_maquinas(maquina_post_eliminar_producto, descompuestas)

-----------------------------------------------------------------------------------------------------
M1  | maquina descompuesta
-----------------------------------------------------------------------------------------------------
M2  |B|B|B|B|I|I|I|I|J|J|J|J|K|K|K|K|L|L|L|L|M|M|M|M|P|P|P|P|P|P|P|P|S|S|S|S|U|U|U|U|Y|Y|Y|Y| | | | |
-----------------------------------------------------------------------------------------------------
M3  | | | | | | | | | | | | | | | | | | | | | | | | | |O|O|O|O|O|Q|Q|Q|Q|Q|V|V|V|V|V|W|W|W|W|W| | | |
-----------------------------------------------------------------------------------------------------
M4  | | | | |A|A| | |I|I| | |J|J| | |K|K| | |L|L| | |M|M| | |N|N|O|O|R|R| | |R|R| | |U|U| | |X|X| | |
-----------------------------------------------------------------------------------------------------
M5  | | | | |B|B|B| |D|D|D| |E|E|E| |F|F|F| |G|G|G| |H|H|H| |P|P|P| |P|P|P|Q|Q|Q| | |T|T|T| | |W|W|W|
-------------------------------------------------------

Visualizar productos afectados

In [170]:
pedidos_a_reprogramar

['A', 'R', 'T', 'F', 'D', 'G', 'H', 'X', 'N', 'E']

Reprogramar lo que se perdio al romperse la maquina 1

In [171]:
maquina_post = maquina_post_romper

for key in pedidos_a_reprogramar:
    pedido = pedidos_dict[key]
    maquina_og = copy.deepcopy(maquina_post)  # Deep copy
    maquina_previo = copy.deepcopy(maquina_post)  # Deep copy
    posibles, litros = datos_pedido(pedido[3])
    codigo_pedido = list(filter(lambda x: pedidos_dict[x] == pedido, pedidos_dict))[0]
    print(codigo_pedido, pedido)
    print(litros,"Litros")
    prod = pedido[3].split("*")[0]
    maquina_post = try_add(maquina_previo, posibles, litros, codigo_pedido, prod, descompuestas)
    if maquina_post == maquina_og:
        print(f"\nNo se logro agregar {pedido} ({codigo_pedido})\n")
        wait_list.append(pedido)
    else:
        despliegue_de_maquinas(maquina_post, descompuestas)

A ['P1', 1.6520154424108469, 'SaveMore', 'P5*40_500ml']
20 Litros
Cooking: M3 (5, 9)
Bottling: M4 ([10, 11])
-----------------------------------------------------------------------------------------------------
M1  | maquina descompuesta
-----------------------------------------------------------------------------------------------------
M2  |B|B|B|B|I|I|I|I|J|J|J|J|K|K|K|K|L|L|L|L|M|M|M|M|P|P|P|P|P|P|P|P|S|S|S|S|U|U|U|U|Y|Y|Y|Y| | | | |
-----------------------------------------------------------------------------------------------------
M3  |Z|Z|Z|Z|Z|A|A|A|A|A| | | | | | | | | | | | | | | |O|O|O|O|O|Q|Q|Q|Q|Q|V|V|V|V|V|W|W|W|W|W| | | |
-----------------------------------------------------------------------------------------------------
M4  | | | | | | | | |I|I|A|A|J|J| | |K|K| | |L|L| | |M|M| | | | |O|O| | | | | | | | |U|U| | | | | | |
-----------------------------------------------------------------------------------------------------
M5  | | | | |B|B|B| | | | | | | | | | | | | | | 